In [3]:
import pandas as pd
import numpy as np
# from transformers import BertModel, BertTokenizer
from sklearn.metrics.pairwise import cosine_similarity
import torch
import json
import os
# from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import csv
import tqdm
# from sentence_transformers import SentenceTransformer
import re
from collections import defaultdict
from time import strftime

In [2]:
import ast
def process_final_raw_res(df):
    # Identify all categories automatically
    ids = set()
    for col in df.columns:
        if col.endswith('_score'):
            ids.add(col.replace('_score', ''))
    
    # Prepare a list for output
    rows = []
    
    for idx, row in df.iterrows():
        location_cat = row['location cat']
        for id_ in ids:
            # Extract lists
            try:
                phrases = ast.literal_eval(row[id_])
                scores = ast.literal_eval(row[f'{id_}_score'])
                pvals = ast.literal_eval(row[f'{id_}_pvals'])

    #             scores = [float(x) for x in ast.literal_eval(row[f'{id_}_score'])]
    #             pvals = [float(x) for x in ast.literal_eval(row[f'{id_}_pvals'])]
            except Exception as e:
                print(f"Error parsing row {idx}, category {id_}: {e}")
                continue

            for phrase, score, pval in zip(phrases, scores, pvals):
                if score > 0 and pval <= 0.05:
                    rows.append({
                        'location cat': location_cat,
                        'identity': id_,
                        'phrase': phrase,
                        'score': score,
                        'pval': pval
                    })
    
    # Create DataFrame and save
    out_df = pd.DataFrame(rows)
    return out_df

In [ ]:
import os
from glob import glob

concept_dir = os.path.join('FINAL_RAW_RES')
if not os.path.exists(concept_dir):
    os.mkdir(concept_dir)

input_folder = 'SUMM_COMBINE_REMOVE_DUP_&CALC' 

# Find all CSV files in the input folder
csv_files = glob(os.path.join(input_folder, '*.csv'))

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    out_df = process_final_raw_res(df)
    out_filename = os.path.join('FINAL_RAW_RES', os.path.basename(csv_file))
    out_df.to_csv(out_filename, index=False)